---
title: Preprocessing
description: Preprocessing the data for future use
---

In [79]:
import pandas as pd
from IPython.display import display, HTML

### Preprocessing match results

In [80]:
# Column names : 'League', 'Country', 'Season', 'Date', 'Home', 'HomeGoals', 'Away', 'AwayGoals'
match_results = pd.read_csv('data/extracted_match_results.csv', parse_dates=['Date'])

# Fix encoding issue : renaming 'Fu\303\237ball-Bundesliga' to 'Bundesliga'
match_results['League'] = match_results['League'].replace('Fu<U+00C3><U+009F>ball-Bundesliga', 'Bundesliga')

# Map country codes to country names
match_country = {'ENG': 'England', 'ITA': 'Italy', 'FRA': 'France', 'GER': 'Germany', 'ESP': 'Spain'}
match_results['Country'] = match_results['Country'].map(match_country)

In [81]:
#| label: match_results
match_results.head()

,League,Country,Season,Date,Home,HomeGoals,Away,AwayGoals
0,Premier League,England,2018,2017-08-11,Arsenal,4.0,Leicester City,3.0
1,Premier League,England,2018,2017-08-12,Watford,3.0,Liverpool,3.0
2,Premier League,England,2018,2017-08-12,Crystal Palace,0.0,Huddersfield,3.0
3,Premier League,England,2018,2017-08-12,West Brom,1.0,Bournemouth,0.0
4,Premier League,England,2018,2017-08-12,Chelsea,2.0,Burnley,3.0


Since we are not interested in match opponents but rather individual team's result, we will modify this dataframe by splitting the match results into two separate rows, one for each team. This will allow us to calculate the statistics for each team separately.

In [82]:
def return_result(goal1, goal2):
    if goal1 > goal2:
        return 'win'
    elif goal1 < goal2:
        return 'loss'
    else:
        return 'draw'
    
match_results['HomeResult'] = match_results.apply(lambda x: return_result(x['HomeGoals'], x['AwayGoals']), axis=1)
match_results['AwayResult'] = match_results.apply(lambda x: return_result(x['AwayGoals'], x['HomeGoals']), axis=1)

home_results = match_results[['League', 'Country', 'Date', 'Home', 'HomeGoals', 'HomeResult']]
home_results = home_results.rename(columns={'Home': 'Team', 'HomeGoals': 'Goals', 'HomeResult': 'Result'})
home_results['isHome'] = True

away_results = match_results[['League', 'Country', 'Away', 'Date', 'AwayGoals', 'AwayResult']]
away_results = away_results.rename(columns={'Away': 'Team', 'AwayGoals': 'Goals', 'AwayResult': 'Result'})
away_results['isHome'] = False

match_results = pd.concat([home_results, away_results], ignore_index=True)

### Preprocessing head coach

In [83]:
#| label: head_coach

# Column names : 'Team', 'League', 'Country', 'HeadCoach', 'Appointed', 'EndDate', 'DaysInPost', 'Matches', 'Wins', 'Draws', 'Losses'
head_coach = pd.read_csv('data/extracted_head_coach.csv', parse_dates=['Appointed', 'EndDate'])
head_coach.head()

,Team,League,Country,HeadCoach,Appointed,EndDate,Tenure,Matches,Wins,Draws,Losses
0,Manchester City,Premier League,England,Pep Guardiola,2016-07-01,NaT,2838,461,341,56,64
1,Manchester City,Premier League,England,Manuel Pellegrini,2013-07-01,2016-06-30,1095,166,101,27,38
2,Manchester City,Premier League,England,Roberto Mancini,2009-12-19,2013-05-13,1241,191,113,38,40
3,Manchester City,Premier League,England,Mark Hughes,2008-06-04,2009-12-19,563,77,37,15,25
4,Manchester City,Premier League,England,Sven-Göran Eriksson,2007-07-06,2008-06-02,332,45,19,11,15


We need to filter head coach that were not active between 2018 to 2022.

In [84]:
# Keep head coach that were appointed before 2022
head_coach = head_coach[head_coach['Appointed'] <= '2022-12-31']
# Keep head coach that were dimissed after 2018 or that are still active
head_coach = head_coach[(head_coach['EndDate'] >= '2018-01-01') | (head_coach['EndDate'].isna())]

:::{caution}
Head coach appointments records, extracted from TransferMarkt, contains data related to head coach in that specific club : tenure, number of matches played, number of matches won, etc. Those datapoint goes beyond our cut-off date (2022-12-31).

One important thing is that those feature beyond cut-off date still relate to a head coach appointment we have in our records. This guarantees us that metrics such *number of club head coach managed* are properly reflected and still relate to head coach performance. 

However, this create an asymetry in our data, as certain data point are limited by a time-frame and others not.
Moreover, we must be careful in how we compare these datapoint to others dataset such as match results as it could easily bias our statistical study.

The only way we combine this dataset to match result is by extracting head coach tenure on day of a match. This does not bias our statistical study as it is a feature that is properly reflected by our cut-off date.
:::

:::{note}
I have considered computing Head Coach performance metric from match results but we would lose information on prior records as well as creating imbalance data for plot such as linear regression of head coach performance over head coach tenure : a long standing coach which would not been dismissed soon after our start date would have a lower number of matches, thus a performance metric with higher variance that would bias linear regression due to long tenure.
:::

We need to ensure that data is coherent and that there is only 1 head coach at a time for a team.

In [85]:
# We need to verify that for a given team there is only 1 head coach at a time
# Each row contains a record of an head coach appointments. This appointment is ongoing between the Appointed and EndDate

head_coach_bis = head_coach.copy()
# Sort data by 'Team' and 'Appointed'
head_coach_bis = head_coach_bis.sort_values(['Team', 'Appointed'])
# Fillna with the last date of the dataset
head_coach_bis['EndDate'] = head_coach_bis['EndDate'].fillna('2022-12-31')
# Check if the next appointment is overlapping with the current one
head_coach_bis['Overlap'] = head_coach_bis.groupby('Team')['Appointed'].shift(-1) < head_coach_bis['EndDate']

# Show team with overlapping appointments
overlapping = head_coach_bis[head_coach_bis['Overlap']]

There is {eval}`overlapping.shape[0]` inconsistent record of head coach in {eval}`' ,'.join(overlapping['team'].unique())` teams.

In [86]:
#| label: hc_inconsistency
head_coach_bis[head_coach_bis['Team'] == 'Stade Reims'][['Team', 'Appointed', 'EndDate', 'Overlap']]

,Team,Appointed,EndDate,Overlap
3497,Stade Reims,2017-05-22,2021-05-25,True
3496,Stade Reims,2018-07-01,2019-03-30,False
3495,Stade Reims,2021-06-23,2022-10-13,False
3494,Stade Reims,2022-10-13,2022-12-31,False


In [87]:
# Filter out overlapping record
head_coach = head_coach[~head_coach.index.isin(overlapping.index)]

Next we would like to add to each head coach record the number of appointment he is completing.

In [88]:
# Using cronological information about appointment
# Add a column to head coach records that tells us about how many appointment head coach has done

head_coach = head_coach.sort_values(['HeadCoach', 'Appointed'])
head_coach['AppointmentNumber'] = head_coach.groupby('HeadCoach').cumcount() + 1
head_coach[head_coach['HeadCoach'].isin(head_coach[head_coach['AppointmentNumber'] > 3]['HeadCoach'])].sort_values('HeadCoach').head()

,Team,League,Country,HeadCoach,Appointed,EndDate,Tenure,Matches,Wins,Draws,Losses,AppointmentNumber
1882,US Sassuolo,Serie A,Italy,Giuseppe Iachini,2017-11-27,2018-06-05,190,26,9,8,9,1
2388,FC Empoli,Serie A,Italy,Giuseppe Iachini,2018-11-06,2019-03-13,127,16,4,4,8,2
1743,ACF Fiorentina,Serie A,Italy,Giuseppe Iachini,2019-12-23,2020-11-09,322,31,12,10,9,3
1741,ACF Fiorentina,Serie A,Italy,Giuseppe Iachini,2021-03-24,2021-06-30,98,10,2,5,3,4
2684,Eintracht Frankfurt,Bundesliga,Germany,Niko Kovac,2016-03-08,2018-06-30,844,91,41,17,33,1


In [89]:
# Check if total_matches = wins + draws + losses
head_coach[head_coach['Matches'] != head_coach['Wins'] + head_coach['Draws'] + head_coach['Losses']].shape[0]

0

In [90]:
# Display Head Coach with lowest number of match
display(head_coach.sort_values('Matches').head(20))
# Remove head coach records with less than 2 matches 
head_coach = head_coach[head_coach['Matches'] > 1]

,Team,League,Country,HeadCoach,Appointed,EndDate,Tenure,Matches,Wins,Draws,Losses,AppointmentNumber
3496,Stade Reims,Ligue 1,France,Sébastien Desmazeau,2018-07-01,2019-03-30,272,0,0,0,0,1
2667,TSG 1899 Hoffenheim,Bundesliga,Germany,Marcel Rapp,2020-06-25,2020-06-30,5,1,1,0,0,1
2013,Udinese Calcio,Serie A,Italy,Igor Tudor,2018-04-24,2018-06-30,67,4,2,1,1,1
1151,Deportivo Alavés,LaLiga,Spain,Juan Muñiz,2020-07-05,2020-07-20,15,4,1,1,2,1
2908,VfB Stuttgart,Bundesliga,Germany,Nico Willig,2019-05-06,2019-06-30,55,4,1,3,0,1
1544,AC Milan,Serie A,Italy,Marco Giampaolo,2019-06-19,2019-10-08,111,7,3,0,4,1
2098,Genoa CFC,Serie A,Italy,Ivan Juric,2018-10-09,2018-12-06,58,8,0,3,5,1
1146,Deportivo Alavés,LaLiga,Spain,Julio Velázquez,2022-04-05,2022-06-30,86,8,3,0,5,2
1115,Getafe CF,LaLiga,Spain,Michel,2021-07-01,2021-10-04,95,8,0,1,7,1
3347,FC Nantes,Ligue 1,France,Miguel Cardoso,2018-06-13,2018-10-02,111,8,1,3,4,1


#### Joining head coach with match results

We would like to add information about how long head coach has been in charge of the team when the match was played. This will allow us to see if the head coach's tenure have any impact on the match result.

However, when trying to join the two dataframes, we found that team names are not consistent between the two dataframes. We will need to fix this before we can join the two dataframes.

In [91]:
# Compute number of team that are in head_coach but not in match_results
coach_teams = set(head_coach['Team'])
match_teams = set(match_results['Team'])

coach_team_not_in_match = coach_teams - match_teams
match_team_not_in_coach = match_teams - coach_teams

len(coach_team_not_in_match), len(match_team_not_in_coach)

(63, 132)

In total, match_results dataset contains {eval}`len(match_team)` teams and head_coach dataset contains {eval}`len(coach_team)` teams. However some teams name are different between the two datasets. For example 'Liverpool' in match_results is 'Liverpool FC' in head_coach. This is problematic as we will need to join data on team's columns.

In total, there is {eval}`len(coach_team_not_in_match)` teams present in head coach records that are not in match results and {eval}`len(match_team_not_in_coach)` teams present in match results but not in head coach records.

We will use Levenshtein distance to find the closest team of *match_results* that match each team in head coach records. We will then manually check the results to ensure that the matches are correct.

In [92]:
from thefuzz import process

team_name_mapping = {}

for coach_team in coach_teams:
    matching_scores = process.extract(coach_team, match_teams, limit=1)
    
    if len(matching_scores) != 0 and matching_scores[0][1] >= 60:
        team_name_mapping[coach_team] = matching_scores[0][0]
    else:
        team_name_mapping[coach_team] = None
        print(f"No match found for {coach_team}")

In [93]:
#| label: team_match_table

name_match = pd.DataFrame(team_name_mapping.items(), columns=['Team in head coach records', 'Team in match results'])
display(HTML(name_match.head().to_html(index=False)))

Team in head coach records,Team in match results
Torino FC,Torino
Cagliari Calcio,Cagliari
Girona FC,Girona
RB Leipzig,RB Leipzig
Athletic Bilbao,Athletic Club


In [94]:
# Map head_coach['team'] with name_match
head_coach['Team'] = head_coach['Team'].map(team_name_mapping)
head_coach

,Team,League,Country,HeadCoach,Appointed,EndDate,Tenure,Matches,Wins,Draws,Losses,AppointmentNumber
1153,Torino,LaLiga,Spain,Abelardo,2017-12-01,2019-06-30,576,69,28,14,27,1
1149,Torino,LaLiga,Spain,Abelardo,2021-01-12,2021-04-05,83,12,1,2,9,2
2863,Mainz 05,Bundesliga,Germany,Achim Beierlorzer,2019-11-18,2020-09-28,315,26,9,4,13,1
2683,Eint Frankfurt,Bundesliga,Germany,Adi Hütter,2018-07-01,2021-06-30,1095,141,67,31,43,1
2742,M'Gladbach,Bundesliga,Germany,Adi Hütter,2021-07-01,2022-06-30,364,37,14,9,14,2
...,...,...,...,...,...,...,...,...,...,...,...,...
439,Barcelona,LaLiga,Spain,Xavi,2021-11-06,2024-06-30,967,133,84,23,26,1
593,Real Madrid,LaLiga,Spain,Zinédine Zidane,2016-01-04,2018-05-31,878,149,105,28,16,1
590,Real Madrid,LaLiga,Spain,Zinédine Zidane,2019-03-11,2021-06-30,842,114,69,25,20,2
1004,Celta Vigo,LaLiga,Spain,Óscar García,2019-11-04,2020-11-09,371,38,8,17,13,1


We can now add head coach days in post to match results.

In [95]:
#| label: final_match_results

# Merge head_coach with match_results
match_results_bis = match_results.merge(head_coach[['Team', 'HeadCoach', 'Appointed', 'EndDate']], on=['Team'], how='left')
# Put aside team that don't have a head coach
no_headcoach = match_results_bis[match_results_bis['HeadCoach'].isna()]
match_results_bis = match_results_bis[~match_results_bis['HeadCoach'].isna()]
# Filter match_results_bis to keep only head coach that were appointed before the match and with no end date or end date after the match
match_results_bis = match_results_bis[(match_results_bis['Date'] >= match_results_bis['Appointed']) & ((match_results_bis['EndDate'].isna()) | (match_results_bis['EndDate'] >= match_results_bis['Date']))]
# Join back the team that don't have a head coach
match_results_bis = pd.concat([match_results_bis, no_headcoach], ignore_index=True)
# Compute daysInPost
match_results_bis['DaysInPost'] = (match_results_bis['Date'] - match_results_bis['Appointed']).dt.days
match_results_bis = match_results_bis.drop(columns=['Appointed', 'EndDate'])
match_results_bis.head()

,League,Country,Date,Team,Goals,Result,isHome,HeadCoach,DaysInPost
0,Premier League,England,2017-08-11,Arsenal,4.0,win,True,Arsène Wenger,7619.0
1,Premier League,England,2017-08-12,Chelsea,2.0,loss,True,Antonio Conte,407.0
2,Premier League,England,2017-08-12,Brighton,0.0,loss,True,Chris Hughton,955.0
3,Premier League,England,2017-08-13,Newcastle Utd,0.0,loss,True,Rafael Benítez,520.0
4,Premier League,England,2017-08-13,Manchester Utd,4.0,win,True,José Mourinho,408.0


### Saving preprocessed data

In [96]:
# Save match_results
match_results_bis.to_csv('data/match_results.csv', index=False)
head_coach.to_csv('data/head_coach.csv', index=False)